In [1]:
from elasticsearch import Elasticsearch

In [148]:
client = Elasticsearch(
    "http://localhost:9200",
    api_key="OFN2Mno1TUJDejVTWHlHeGlyM2w6LTExemZTQ2xRbE9MVEtYT1kzTjc4UQ=="
)

In [5]:
client.indices.create(index="test_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [6]:
client.index(
    index="test_index",
    id="test_id",
    document={
        "this": "that",
        "the": "other",
    }
)

ObjectApiResponse({'_index': 'test_index', '_id': 'test_id', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [7]:
doc = client.get(index="test_index", id="test_id")

In [8]:
print(doc)

{'_index': 'test_index', '_id': 'test_id', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'this': 'that', 'the': 'other'}}


In [9]:
import sqlite3

In [10]:
con = sqlite3.connect("../db/vb-se.db")

In [11]:
cur = con.cursor

In [48]:
query = """ 
select 
    v.video_id,
    v.title,
    v.url,
    v.thumbnail,
    v.description,
    v.channel_id,
    v.channel_name,
    vtd.transcript,
    v.upload_date
from videos v join video_text_data vtd on vtd.video_id = v.video_id limit 10
"""

In [86]:
res = con.execute(query)

In [87]:
result_data_all = res.fetchall()

In [88]:
result_data_all[0]

('WSDRF7VGmIs',
 'Understanding Boko Haram',
 'https://www.youtube.com/watch?v=WSDRF7VGmIs',
 'https://i.ytimg.com/vi/WSDRF7VGmIs/maxresdefault.jpg',
 "In which John discusses Nigeria, its complicated history, and how that history has shaped the region from which the Islamist terrorist group Boko Haram emerged. Also discussed is the history of Boko Haram itself and its recent atrocities.\n\nAnd here is your standard reminder that educational videos are allowed to be over 4 minutes long.\n\nSome links for further reading:\n\nA basic overview of Boko Haram from the bbc: http://www.bbc.com/news/world-africa-13809501\nA timeline of Boko Haram's attacks: http://en.wikipedia.org/wiki/Timeline_of_Boko_Haram_insurgency\nA detailed introduction to Boko Haram: http://en.wikipedia.org/wiki/Boko_Haram\nA good overview of how Islam came to sub-saharan Africa: http://www.dhspriory.org/kenny/Sist.htm\nMore info on the 1967-1970 Nigerian Civil War: http://en.wikipedia.org/wiki/Nigerian_Civil_War\n\n--

In [89]:
def get_doc(result_data):
    return {
        "video_id": result_data[0],
        "title": result_data[1],
        "url": result_data[2],
        "thumbnail": result_data[3],
        "description": result_data[4],
        "channel_id": result_data[5],
        "channel_name": result_data[6],
        "transcript": result_data[7],
        "upload_date": result_data[8],
    }

In [90]:
res_docs = list(map(get_doc, result_data_all))

In [91]:
res_docs[0]

{'video_id': 'WSDRF7VGmIs',
 'title': 'Understanding Boko Haram',
 'url': 'https://www.youtube.com/watch?v=WSDRF7VGmIs',
 'thumbnail': 'https://i.ytimg.com/vi/WSDRF7VGmIs/maxresdefault.jpg',
 'description': "In which John discusses Nigeria, its complicated history, and how that history has shaped the region from which the Islamist terrorist group Boko Haram emerged. Also discussed is the history of Boko Haram itself and its recent atrocities.\n\nAnd here is your standard reminder that educational videos are allowed to be over 4 minutes long.\n\nSome links for further reading:\n\nA basic overview of Boko Haram from the bbc: http://www.bbc.com/news/world-africa-13809501\nA timeline of Boko Haram's attacks: http://en.wikipedia.org/wiki/Timeline_of_Boko_Haram_insurgency\nA detailed introduction to Boko Haram: http://en.wikipedia.org/wiki/Boko_Haram\nA good overview of how Islam came to sub-saharan Africa: http://www.dhspriory.org/kenny/Sist.htm\nMore info on the 1967-1970 Nigerian Civil Wa

In [57]:
client.indices.create(
    index="vb-se-test",
    mappings={
        "properties": {
            "semantic_text": {
                "type": "semantic_text",
            },
            "content": {
                "type": "text",
                "copy_to": "semantic_text"
            },
            "video_id": {
                "type": "text"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vb-se-test'})

In [68]:
client.index(
    index="vb-se-test",
    id=res_docs[0]["video_id"],
    document={
        "content": res_docs[0]["transcript"],
        "video_id": res_docs[0]["video_id"],
    }
)

ObjectApiResponse({'_index': 'vb-se-test', '_id': 'WSDRF7VGmIs', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [69]:
resp = client.search(
    index="vb-se-test",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "content": "guild"
                            }
                        }
                    }
                },
                {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "semantic_text",
                                "query": "guild"
                            }
                        }
                    }
                }
            ]
        }
    },
)

In [96]:
resp['hits']['hits'][0]["_source"]["video_id"]

'WSDRF7VGmIs'

In [149]:
client.ingest.put_pipeline(
    id="ingest_with_dates",
    processors=[
        {
            "set": {
                "field": "created_at",
                "value": "{{_ingest.timestamp}}"
            }
        }
    ]
)

ObjectApiResponse({'acknowledged': True})

In [150]:
client.indices.create(
    index="vb-se-videos",
    mappings={
        "properties": {
            "semantic_text": {
                "type": "semantic_text",
            },
            "transcript": {
                "type": "text",
                "copy_to": "semantic_text"
            },
            "title": {
                "type": "text",
                "copy_to": "semantic_text"
            },
            "description": {
                "type": "text",
                "copy_to": "semantic_text"
            },
            "video_id": {
                "type": "text"
            },
            "channel_id": {
                "type": "text"
            },
            "thumbnail": {
                "type": "text"
            },
            "url": {
                "type": "text"
            },
            "upload_date": {
                "type": "date"
            },
            "channel_name": {
                "type": "text"
            },
            "created_at": {
                "type": "date"
            }
        }
    },
    settings={
        "index.default_pipeline": "ingest_with_dates"
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vb-se-videos'})

In [107]:
client.indices.create(
    index="vb-se-channels",
    mappings={
        "properties": {
            "channel_id": {
                "type": "text"
            },
            "channel_name": {
                "type": "text"
            }
        }
    }
)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [vb-se-channels/4ZSB5ga4R2mE1RQQxXprZw] already exists')

In [103]:
client.index(
    index="vb-se-channels",
    document={
        "channel_id": "UCGaVdbSav8xWuFWTadK6loA",
        "channel_name": "vlogbrothers",
    }
)

ObjectApiResponse({'_index': 'vb-se-channels', '_id': 'cCtn0pMBCz5SXyGxDb7T', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [145]:
resp = client.search(
    index="vb-se-videos",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "content": "swans swimming"
                            }
                        }
                    }
                },
                {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "semantic_text",
                                "query": "swans swimming"
                            }
                        }
                    }
                }
            ]
        }
    },
    fields=[
        "video_id",
        "title"
    ]
)

In [147]:
resp['hits']['hits'][0]["_source"]

{'channel_name': 'vlogbrothers',
 'thumbnail': 'https://i.ytimg.com/vi/XJIbiowBfIs/maxresdefault.jpg',
 'transcript': " good morning J and Merry Christmas I have good news we have finally come up with and I have a penny here because I was doing calculations the Charities that are being funded through the project for awesome we have coming in fifth place with 915 votes the kind Foundation kids in need of death thanks to an amazing video Dark Horse candidate out of nowhere really fantastic work in fourth place with 1,43 votes we have kea.org now there are two things going on with Kea at any given time one running the actual service of Kaa allowing people to connect with entrepreneurs in the developing world two we have the actual entrepreneurs who can be funded not through donations but through loans and so what we're going to do here is we're going to split that in half half of it's going to go to running and maintaining kea.org and the other half is going to go into loans for entrepren